In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import cuda
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, sampler
import torch.nn.functional as F
from torch_utils import AverageMeter
import math
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from numpy import inf
from merge_datasets import MergeDatasets
from merge_features import MergeFeatures
from sklearn.preprocessing import StandardScaler
from joblib import load

from sklearn.model_selection import train_test_split
from glob import glob
from torchvision import transforms
from torchvision import datasets
from torchvision import models
from torch import optim, cuda, Tensor

import numpy as np
import os
from PIL import Image
from timeit import default_timer as timer
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

In [2]:
# define multi-layer perceptron deep neural network model
# network includes two linear layers, one Relu layer, and one dropout layer
class MLP(nn.Module):
    def __init__(self, input_d, hidden_d, output_d):
        super().__init__()

        self.linear1 = nn.Linear(input_d, hidden_d)
        self.linear2 = nn.Linear(hidden_d, output_d)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):

        x = self.linear1(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.linear2(x)

        return x

In [3]:
input_d = 13
hidden_d = 64
output_d = 1

model = MLP(input_d, hidden_d, output_d)
model.load_state_dict(torch.load('model_state.pth'))
model.eval()

<All keys matched successfully>

MLP(
  (linear1): Linear(in_features=13, out_features=64, bias=True)
  (linear2): Linear(in_features=64, out_features=1, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [4]:
co2_df = pd.read_csv('../predictions/co2_concentrations_predictions_2024_2124.csv')
forest_df = pd.read_csv('../predictions/forest_area_predictions_2024_2124.csv')
land_df = pd.read_csv('../predictions/land_use_predictions_2024_2124.csv')
temp_df = pd.read_csv('../predictions/temperature_anomaly_predictions_2024_2124.csv')

merged_features = MergeFeatures(co2_df, forest_df, land_df, temp_df)
merged_df = merged_features.merge()

/Users/jackzhang/anaconda3/envs/cs4262/lib/python3.10/site-packages/pycountry/db.py:51: UserWarning: Country's official_name not found. Country name provided instead.
  warnings.warn(warning_message, UserWarning)
/Users/jackzhang/anaconda3/envs/cs4262/lib/python3.10/site-packages/pycountry/db.py:51: UserWarning: Country's common_name not found. Country name provided instead.
  warnings.warn(warning_message, UserWarning)


In [5]:
scaler_X = load('scaler_X.bin')
scaler_y = load('scaler_y.bin')

X_pred_continuous = merged_df[['Year', 'Predicted CO2', 'Forest area', 'Land use: Built-up area', 'Land use: Grazingland', 'Land use: Cropland', 'Temperature anomaly']].values
entities_pred = pd.get_dummies(merged_df['Entity'])
unique_entities = ['Africa', 'Asia and Pacific', 'Europe and Central Asia', 'Latin America and the Caribbean', 'North America', 'World']
entities_pred = pd.get_dummies(merged_df['Entity'], columns=unique_entities)

for entity in unique_entities:
    if entity not in entities_pred.columns:
        entities_pred[entity] = 0

entities_pred = entities_pred[unique_entities]
# Merge continuous and categorical features
X_pred = np.hstack([X_pred_continuous, entities_pred])

# Ensure the number of features in X_pred matches what scaler_X expects
X_pred_scaled = scaler_X.transform(X_pred)

X_pred_tensor = torch.tensor(X_pred_scaled, dtype=torch.float32)

model.eval()
with torch.no_grad():
    lpi_pred_scaled = model(X_pred_tensor).numpy()

lpi_predictions = scaler_y.inverse_transform(lpi_pred_scaled.reshape(-1, 1)).flatten()
lpi_predictions_adjusted = np.maximum(lpi_predictions, 0)

predictions_df = pd.DataFrame({
    'Year': np.arange(2024, 2125),
    'Predicted_LPI': lpi_predictions_adjusted
})
predictions_df.to_csv('predicted_lpi_2024_2124.csv', index=False)

MLP(
  (linear1): Linear(in_features=13, out_features=64, bias=True)
  (linear2): Linear(in_features=64, out_features=1, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)